Figuring out how to read/organize my files

In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [2]:
archive = pd.read_csv('data/fer2013.csv')
archive['image'] = archive['pixels'].apply(lambda x: np.array(x.split()).astype(int).reshape(48,48))

In [3]:
# plt.bar(range(7),archive.emotion[archive.Usage == 'Training'].value_counts().sort_index())

In [4]:
# plt.bar(range(7),archive.emotion[archive.Usage == 'PublicTest'].value_counts().sort_index())

In [5]:
# plt.bar(range(7),archive.emotion[archive.Usage == 'PrivateTest'].value_counts().sort_index())

In [6]:
em_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
em_indices = []
for i in range(7):
    em_indices.append(archive[archive['emotion']==i].index)

In [7]:
# fig,ax = plt.subplots(7,10, figsize=(20,14))
# [ax.set_axis_off() for ax in ax.ravel()]
# for j in range(7):
#     for axi, i in enumerate(em_indices[j][:10]):
#         ax[j,axi].imshow(archive.image[i],cmap='gray')
#         ax[j,axi].set_title(em_labels[j])

In [8]:
import tensorflow as tf

In [9]:
x_train = np.stack(archive.image[archive.Usage == 'Training'].to_numpy()).reshape(-1, 48, 48, 1)/255
y_train = archive.emotion[archive.Usage == 'Training'].to_numpy()
x_test = np.stack(archive.image[archive.Usage == 'PublicTest'].to_numpy()).reshape(-1, 48, 48, 1)/255
y_test = archive.emotion[archive.Usage == 'PublicTest'].to_numpy()

In [10]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', input_shape=(48, 48, 1)),
#     tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
#     tf.keras.layers.MaxPool2D(),
#     tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),    
#     tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(7, activation='softmax')
# ])
# model.summary()

In [11]:
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [12]:
# model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

In [13]:
# x_pred = model.predict(x_test).argmax(axis=1)

In [15]:
# plt.bar(range(7), archive.emotion[archive.Usage == 'PublicTest'][x_pred != y_test].value_counts().sort_index())

In [16]:
# model.save('model/model_0.4')

In [17]:
# test_img = cv2.imread('src/news_preview_mob_image__preview_404.jpg')
# face_cascade = cv2.CascadeClassifier('src/haarcascade_frontalface_default.xml')
# faces = face_cascade.detectMultiScale(test_img, 1.1, 4)
# plt.imshow(test_img)

In [18]:
def get_face(img, face_coord):
    return img[face_coord[1]:face_coord[1]+face_coord[3],face_coord[0]:face_coord[0]+face_coord[2]]

In [19]:
# for (x, y, w, h) in faces:
#     cv2.rectangle(test_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
#     cv2.putText(test_img, 'Emotion', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))

In [20]:
x_train_3channel = np.zeros((x_train.shape[0], 48, 48, 3))
x_test_3channel = np.zeros((x_test.shape[0], 48, 48, 3))
for idx, image in enumerate(x_train):
    x_train_3channel[idx] = np.dstack((image, image, image))
for idx, image in enumerate(x_test):
    x_test_3channel[idx] = np.dstack((image, image, image))

In [21]:
densenet = tf.keras.applications.DenseNet121(include_top=False, input_shape=(48, 48, 3), classes=7, weights='imagenet')
densenet.summary()

_conv[0][0]        
__________________________________________________________________________________________________
conv5_block4_0_bn (BatchNormali (None, 1, 1, 608)    2432        conv5_block3_concat[0][0]        
__________________________________________________________________________________________________
conv5_block4_0_relu (Activation (None, 1, 1, 608)    0           conv5_block4_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block4_1_conv (Conv2D)    (None, 1, 1, 128)    77824       conv5_block4_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block4_1_bn (BatchNormali (None, 1, 1, 128)    512         conv5_block4_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block4_1_relu (Activation (None, 1, 1, 128)    0           conv5_block4_1_bn[0][0] 

In [23]:
densenet.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
densenet.fit(x_train_3channel, y_train)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
densenet.save('model/densenet_0.1')